## Oracle 23ai Vector Store와 OCI GEN AI LLM (Cohere Command R+)을 활용한 RAG 구현

### 1. Oracle Database 23ai 접속

In [28]:
import os
import time
import oracledb
import configparser
config = configparser.ConfigParser()
config.read("oci.env")

username = config["DATABASE"]["USERNAME"]
password = config["DATABASE"]["PASSWORD"]
host = config["DATABASE"]["HOST"]
port = config["DATABASE"]["PORT"]
service_name = config["DATABASE"]["SERVICE_NAME"]
table_name = config["DATABASE"]["TABLE_NAME_CV_LANG"]
compartment_id = config["OCI"]["compartment_id"]
dsn=host+":"+port+"/"+service_name
upstage_api_key=config["APIKEY"]["UPSTAGE_API_KEY"]

try:
    oracledb.init_oracle_client()
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("\nConnected to the Oracle Database 23.4.\n")
except Exception as e:
    print(e)
    print("\nConnection failed!\n")
    


Connected to the Oracle Database 23.4.



### 2. Load the document
#### 텍스트 추출: Oracle Doc Loader, Oracle Text Splitter 사용

In [29]:
from langchain_community.document_loaders.oracleai import OracleTextSplitter 
from langchain_community.document_loaders.oracleai import OracleDocLoader
from langchain_core.documents import Document

pdf_file="/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf"

splitter_params = {"BY" :"words", "MAX": 200, "OVERLAP": 10, "SPLIT": "sentence", "LANGUAGE": "KOREAN", "NORMALIZE": "all"}
splitter = OracleTextSplitter(conn=connection, params=splitter_params)

chunks_with_mdata = []
doc_origin = Document
max_lengh_oracle_allow=9000
counter = 0  
document_num = 0

loader_params = {}        
loader_params['file'] = pdf_file
# instantiate loader, splitter and embedder
loader = OracleDocLoader(conn=connection, params=loader_params)

# read the docs, convert blob docs to clob docs
docs = loader.load()
print(f"Number of docs loaded: {len(docs)}")

for id, doc in enumerate(docs, start=1):
    #remove line break from the text document
    doc.page_content = doc.page_content.replace("\n", "")
    doc_origin.page_content = doc.page_content
    # check the doc
    if len(doc.page_content)>max_lengh_oracle_allow :
        #reduce the text to max_lengh_oracle_allow
        doc.page_content = doc.page_content[:9000]
    document_num += 1
    
    # chunk the doc
    chunks = splitter.split_text(doc_origin.page_content)
    print(f"Doc {id}: chunks# {len(chunks)}")

#For each chunk create chunk_metadata with 
for ic, chunk in enumerate(chunks, start=1):
    counter += 1  
    chunk_metadata = doc.metadata.copy()  
    chunk_metadata['id'] = str(counter)  
    chunk_metadata['document_id'] = str(document_num)
    # chunk_metadata['document_summary'] = str(summ[0])
    chunks_with_mdata.append(Document(page_content=str(chunk), metadata=chunk_metadata))

print(f"Doc {id}: page_content: { chunks_with_mdata[4].page_content} metadata: {chunks_with_mdata[4].metadata}")

Number of docs loaded: 1
Doc 1: chunks# 89
Doc 1: page_content: ··························································6 2. 기업/산업 ▹ 스태빌리티AI, 차세대 이미지 생성 AI '스테이블 디퓨전 3' 프리뷰 버전 공개 ·················7 ▹ 오픈AI, metadata: {'SOURCE MIME TYPE': 'application/pdf', 'creation date': '4/8/2024 12:46:55 AM', 'author': 'spri', 'revision date': '4/8/2024 12:46:55 AM', 'Creator': '\rHwp 2018 10.0.0.13764', 'publisher': 'Hancom PDF 1.3.0.542', 'PDFVersion': '\r1.4', '_oid': '6692c7e26aa918ee066f7ec27b26bfd4', '_file': '/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf', 'id': '5', 'document_id': '1'}


### 3. Embedding 및 벡터 데이터베이스에 입력
#### Embedding Model: OCI GenAI cohere.embed-multilingual-v3.0

In [30]:
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings.oracleai import OracleEmbeddings
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS

embedder = OCIGenAIEmbeddings(
            model_id="cohere.embed-multilingual-v3.0",
            service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
            compartment_id= compartment_id)
distance_strategy=DistanceStrategy.COSINE    #COSINE
table_name_with_strategy = table_name+'_'+distance_strategy

s1time = time.time()
vector_store = OracleVS.from_documents(chunks_with_mdata, embedder, client=connection, table_name=table_name_with_strategy, distance_strategy=distance_strategy)

### Create Oracle HNSW Index
oraclevs.create_index(client=connection,vector_store=vector_store, params={
    "idx_name": "hnsw"+table_name_with_strategy, "idx_type": "HNSW"
})
s2time = time.time()

if vector_store is not None:
    print( f"Documents loading, chunking and generating embeddings are complete.\nVectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")
else:
    print("\nFailed to get the VectorStore populated.\n")

Documents loading, chunking and generating embeddings are complete.
Vectorizing and inserting chunks duration: 2.6 sec.


### 4. Retriever 생성 및 및 유사도 검색
#### Embedding Model: OCI GenAI cohere.embed-multilingual-v3.0

In [31]:
vector_store = OracleVS(client=connection, 
                        embedding_function=embedder, 
                        table_name=table_name_with_strategy, 
                        distance_strategy=distance_strategy)

user_question = ("최초의 AI법은 언제 통과 되었니? 출처나 참고 문서도 같이 알려줘.");

if user_question:
    s1time =  time.time()
    result_chunks = vector_store.similarity_search(user_question)
    s2time = time.time()
    print(f"result_chunks={result_chunks}\nSearch for the user question in the Oracle Database 23ai and return similar chunks duration: {round(s2time - s1time, 1)} sec.")


result_chunks=[Document(metadata={'SOURCE MIME TYPE': 'application/pdf', 'creation date': '4/8/2024 12:46:55 AM', 'author': 'spri', 'revision date': '4/8/2024 12:46:55 AM', 'Creator': '\rHwp 2018 10.0.0.13764', 'publisher': 'Hancom PDF 1.3.0.542', 'PDFVersion': '\r1.4', '_oid': '6692c7e26aa918ee066f7ec27b26bfd4', '_file': '/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf', 'id': '1', 'document_id': '1'}, page_content='2024년 4월호2024년 4월호Ⅰ.인공지능 산업 동향 브리프 1. 정책/법제 ▹ 유럽의회 본회의에서 세계 최초의 AI 법 통과 ·····································································1 ▹ 유럽평의회,'), Document(metadata={'SOURCE MIME TYPE': 'application/pdf', 'creation date': '4/8/2024 12:46:55 AM', 'author': 'spri', 'revision date': '4/8/2024 12:46:55 AM', 'Creator': '\rHwp 2018 10.0.0.13764', 'publisher': 'Hancom PDF 1.3.0.542', 'PDFVersion': '\r1.4', '_oid': '6692c7e26aa918ee066f7ec27b26bfd4', '_file': '/home/opc/23ai_rag_demo/app/pdfs/SPRi_AI_Brief_4.pdf', 'id': '19', 'document_id': '1'}, page_content='·······

### 5. Langchain RAG
#### Vector Store Retriver cohere.command-r-plus LLM 모델 및  사용

In [34]:
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

chat = ChatOCIGenAI(
    model_id="cohere.command-r-plus",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    model_kwargs={"temperature": 0.7, "max_tokens": 500, "top_p": 0.6}
)

message = [
    (
        "system",
        """
        질문-답변 업무를 돕는 AI 어시스턴트입니다. 
        문서의 내용을 참고해서 답변해 주세요.:
        \n\n
        {context}",
        """
    ),
    ("human", "{human}"),
]

prompt = ChatPromptTemplate.from_messages(message)

chain = {
"context": vector_store.as_retriever(search_kwargs={'k':3}),
"human": RunnablePassthrough(),
} | prompt | chat | StrOutputParser()

s1time=time.time()
response = chain.invoke(user_question)
s2time=time.time()

print( f"response={response}\nSend user question and ranked chunks to LLM and get answer duration: {round(s2time - s1time, 1)} sec.")

2024-07-13 18:38:42,778 - ERROR - An unexpected error occurred: ORA-51808: VECTOR_DISTANCE() is not supported for vectors with different dimension counts (4096, 1024).
Help: https://docs.oracle.com/error-help/db/ora-51808/
Traceback (most recent call last):
  File "/home/opc/miniconda3/envs/23ai_rag_demo/lib/python3.12/site-packages/langchain_community/vectorstores/oraclevs.py", line 54, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/opc/miniconda3/envs/23ai_rag_demo/lib/python3.12/site-packages/langchain_community/vectorstores/oraclevs.py", line 722, in similarity_search_by_vector_with_relevance_scores
    cursor.execute(query, embedding=embedding_arr)
  File "/home/opc/miniconda3/envs/23ai_rag_demo/lib/python3.12/site-packages/oracledb/cursor.py", line 701, in execute
    impl.execute(self)
  File "src/oracledb/impl/thick/cursor.pyx", line 306, in oracledb.thick_impl.ThickCursorImpl.execute
  File "src/oracledb/impl/thick/utils.pyx", line 4

RuntimeError: Unexpected error: ORA-51808: VECTOR_DISTANCE() is not supported for vectors with different dimension counts (4096, 1024).
Help: https://docs.oracle.com/error-help/db/ora-51808/

### 6. Langchain RAG
#### SQL Retriver, cohere.command-r-plus LLM 모델 사용

In [33]:
import json

embedder_params = {"provider": "ocigenai", 
                   "credential_name": "YH_OCI_CRED", 
                   "url": "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/embedText", 
                   "model": "cohere.embed-multilingual-v3.0"}

sql_query = f"""SELECT text, metadata 
FROM {table_name_with_strategy}
ORDER BY vector_distance(
    embedding, 
    (
        SELECT TO_VECTOR(et.embed_vector) AS embed_vector 
        FROM DBMS_VECTOR_CHAIN.UTL_TO_EMBEDDINGS(
            :query, 
            JSON(:embedder_params)
        ) t, 
        JSON_TABLE (
            t.column_value, 
            '$[*]' COLUMNS (
                embed_id NUMBER PATH '$.embed_id', 
                embed_data VARCHAR2(4000) PATH '$.embed_data', 
                embed_vector CLOB PATH '$.embed_vector'
            )
        ) et
    )
, COSINE)
FETCH FIRST 3 ROWS ONLY
"""

cur = connection.cursor()
cur.execute(sql_query, {'query': user_question, 'embedder_params': json.dumps(embedder_params)})
rows = cur.fetchall()

documents = [Document(page_content=row[0]) if isinstance(row[0], str) else Document(page_content=row[0].read(), metadata=json.loads(row[1].read())) for row in rows]

context = '\n\n'.join([d.page_content for d in documents])

chat = ChatOCIGenAI(
    model_id="cohere.command-r-plus",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="ocid1.compartment.oc1..aaaaaaaal7ipgtkkohxxjdbgxmqap4jx3gloyd52f33ujv3thz45uwopjmna",
    model_kwargs={"temperature": 0.7, "max_tokens": 500, "top_p": 0.6},
)

message = [
    (
        "system",
        """
        질문-답변 업무를 돕는 AI 어시스턴트입니다. 
        문서의 내용을 참고해서 답변해 주세요.:
        \n\n
        {context}",
        """
    ),
    ("human", "{human}"),
]
prompt = ChatPromptTemplate.from_messages(message)

chain = {
    "context": RunnablePassthrough(),
    "human": RunnablePassthrough()
} | prompt | chat | StrOutputParser()

s1time = time.time()
response = chain.invoke({
    "human": user_question,
    "context": context  # pass context as a string here
})
s2time = time.time()

print( f"response={response}\nSend user question and ranked chunks to LLM and get answer duration: {round(s2time - s1time, 1)} sec.")

response=세계 최초의 AI 법은 2024년 3월 13일 유럽의회 본회의에서 통과되었습니다. 이 법은 EU 회원국의 승인을 얻어 올해 안에 발효될 예정이며, AI로 인한 잠재적 위험으로부터 기본권, 민주주의, 법치, 환경 지속 가능성을 보호하는 것을 목표로 합니다.
Send user question and ranked chunks to LLM and get answer duration: 3.0 sec.
